In [0]:
epochs = 10

# পার্ট 6 - CNN ব্যবহার করে MNIST এর উপরে ফেডারেটেড লার্নিং

## PyTorch + PySyft এর ১০ টি লাইন দিয়ে ফেডারেটেড লার্নিংয়ে আপগ্রেড করুন


### প্রসঙ্গ

ফেডারেটেড লার্নিং একটি অত্যন্ত উত্তেজনাপূর্ণ এবং উৎসাহব্যঞ্জক মেশিন লার্নিং কৌশল যা এমন সিস্টেম তৈরিতে লক্ষ্য করে যা কি-না বিকেন্দ্রীভূত ডেটা থেকে শিখতে পারে। ধারণাটি হলো ডেটাটি তার প্রযোজকের হাতে রয়েছে (যা - কর্মী_নামেও পরিচিত), যা গোপনীয়তা এবং মালিকানা উন্নত করতে সহায়তা করে এবং মডেলটি শ্রমিকদের মধ্যে ভাগ করে নেওয়া হয়েছে। একটি তাৎক্ষণিক অ্যাপ্লিকেশন হলো - টেক্সট লেখার সময় আপনার মোবাইল ফোনে পরবর্তী শব্দটির পূর্বাভাস দেওয়া। সেক্ষেত্রে আপনি চান না আপনার ডাটা প্রশিক্ষণের জন্য ব্যবহৃত হোক। যেমনঃ আপনার বার্তা - কোনও কেন্দ্রীয় সার্ভারে প্রেরণ করা হোক।

ফেডারেটড লার্নিংয়ের উত্থান তথ্যের সাথে গোপনীয়তা সচেতনতার প্রসারের সাথে দৃঢ়ভাবে সংযুক্ত এবং ইইউতে জিডিপিআর যা মে, ২০১৮  সাল থেকে ডেটা সুরক্ষা জারি করে, সেটি অনুঘটক হিসাবে কাজ করেছে। নিয়ন্ত্রণের পূর্বাভাস দেওয়ার জন্য, অ্যাপল বা গুগলের মতো বড় বড় প্রতিষ্ঠান বিশেষত মোবাইল ব্যবহারকারীর গোপনীয়তা রক্ষার জন্য এই প্রযুক্তিতে ব্যাপক বিনিয়োগ শুরু করেছে, তবে তারা তাদের সরঞ্জামগুলি উন্মুক্ত করেনি। OpenMined-এ আমরা বিশ্বাস করি যে যে কেউ মেশিন লার্নিং প্রকল্প পরিচালনা করতে ইচ্ছুক তিনি খুব অল্প প্রচেষ্টা সহ গোপনীয়তা সংরক্ষণের সরঞ্জামগুলি প্রয়োগ করতে সক্ষম হবেন। আমরা এক লাইনে ডেটা এনক্রিপ্ট করার জন্য সরঞ্জাম তৈরি করেছি [আমাদের ব্লগ পোস্টে উল্লিখিত](https://blog.openmined.org/training-cnns-using-spdz/) এবং আমরা এখন নতুন পাইটর্চ 1.0 সংস্করণ ব্যবহার করে আমাদের ফেডারেটেড লার্নিং ফ্রেমওয়ার্ক প্রকাশ করি যা উৎসাহ অর্জন করে সুরক্ষিত এবং স্কেলযোগ্য মডেলগুলি তৈরিতে একটি intuitive ইন্টারফেস সরবরাহ করতে।

এই টিউটোরিয়ালে, আমরা সরাসরি ব্যবহার করব [পাইটর্চ ব্যবহার করে MNIST-তে CNN প্রশিক্ষণের প্রথাগত উদাহরণ](https://github.com/pytorch/example/blob/master/mnist/main.py) এবং দেখাবো আমাদের [PySyft লাইব্রেরি](https://github.com/OpenMined/PySyft/) ব্যবহার করে এর সাথে ফেডারেটেড লার্নিং বাস্তবায়ন করা কতটা সহজ। আমরা উদাহরণের প্রতিটি অংশে যাবো এবং সেই কোডটি আন্ডারলাইন করব যা পরিবর্তিত হয়েছে।

আপনি এই উপাদানগুলি [আমাদের ব্লগপোস্ট](https://blog.openmined.org/upgrade-to-fedrated-learning-in-10-lines) এও পেতে পারেন।

লেখক:
- থিও রাইফেল(Théo Ryffel) - গিটহাব: [@LaRiffle](https://github.com/LaRiffle)

অনুবাদক:
- সায়ন্তন দাস - গিটহাব: [@ucalyptus](https://github.com/ucalyptus)

- মীর মোহাম্মদ জাবের(Mir Mohammad Jaber) - Twitter: [@jabertuhin](https://twitter.com/jabertuhin)

**ঠিক আছে, শুরু করা যাক!**

### ইম্পোর্ট এবং মডেল নির্দিষ্টকরণ

প্রথমে আমরা অফিসিয়াল প্যাকেজসমূহ ইম্পোর্ট করি

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

এবং এরপর পাইসাইফ্ট সম্পর্কিত নির্দিষ্টগুলো প্যাকেজগুলো। বিশেষত আমরা দূরবর্তী কর্মী `alice` এবং `bob`-কে সংজ্ঞায়িত করি

In [0]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

আমরা শেখার কাজটির সেটিংস(settings) সংজ্ঞায়িত করি

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### ডেটা লোড করা এবং কর্মীদের কাছে প্রেরণ
আমরা প্রথমে ডেটা লোড করি এবং প্রশিক্ষণ ডেটাসেটকে। `ফেডারেট` পদ্ধতিটি ব্যবহার করে কর্মীদের জুড়ে ফেডারেশনযুক্ত ডেটাসেট বিভক্তিতে রূপান্তর করি। এই ফেডারেটেড ডেটাসেটটি এখন একটি ফেডারেটেড ডেটা লোডারকে দেওয়া হয়। টেস্ট ডেটাসেট অপরিবর্তিত রয়েছে।

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

### CNN স্পেসিফিকেশন
এখানে আমরা অফিসিয়াল উদাহরণের ঠিক একই সিএনএন ব্যবহার করি।

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### ট্রেন এবং টেস্টের ফাংশনগুলো সংজ্ঞায়িত করুন
ট্রেনের ফাংশনগুলোর জন্য, ডেটা ব্যাচগুলি `alice` এবং `bob` জুড়ে বিতরণ করা হয়েছে, তাই আপনাকে প্রতিটি ব্যাচের জন্য সঠিক জায়গায় মডেলটি প্রেরণ করতে হবে। তারপরে, আপনি স্থানীয় পাইটর্চ যেভাবে ব্যবহার করছেন তেমন একই সিনট্যাক্সের সাহায্যে আপনি সমস্ত ক্রিয়াকলাপ দূরবর্তীভাবে সম্পাদন করেন। আপনার কাজ শেষ হয়ে গেলে, আপনি আপডেট হওয়া মডেলটি এবং এর লস(loss) ফিরে পেয়েছেন এতে আরো উন্নতি খোঁজার জন্য।

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

টেস্ট ফাংশনগুলো বদলায় না!

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### প্রশিক্ষণ চালু করুন!

In [0]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



এ voilà! আপনি এখানে ফেডারেটেড লার্নিং ব্যবহার করে দূরবর্তী ডেটাতে একটি মডেল প্রশিক্ষণ দিয়েছেন!

## শেষ একটি  জিনিস
আমি জানি যে আপনি জিজ্ঞাসা করতে মারা যাচ্ছেন এমন একটি প্রশ্ন রয়েছে: **সাধারণ পাইটর্চের তুলনায় ফেডারেট লার্নিং করতে কত সময় লাগে?**

গণনার সময়টি আসলে **স্বাভাবিক পাইটর্চ সম্পাদনের জন্য ব্যবহৃত** বারের চেয়ে দ্বিগুণ কম! আরও স্পষ্টভাবে, এটি 1.9 গুণ বেশি সময় নেয়, যা আমরা যুক্ত করতে সক্ষম হওয়া বৈশিষ্ট্যের তুলনায় খুব কম।

## উপসংহার

যেমন আপনি পর্যবেক্ষণ করেন, আমরা MNIST অফিসিয়াল পাইটর্চ উদাহরণটিকে বাস্তব ফেডারেট লার্নিং সেটিংয়ে আপগ্রেড করার জন্য ১০ টি লাইন কোড সংশোধন করেছি!

অবশ্যই, আমরা এমন অনেক উন্নতির কথা ভাবতে পারি। আমরা চাই শ্রমিকদের মধ্যে সমান্তরালভাবে গণনা পরিচালিত করতে এবং ফেডারেটেড গড় সম্পাদন করতে, কেন্দ্রীয় মডেলকে প্রতি `n` ব্যাচ পরপর আপডেট করতে, শ্রমিকদের মধ্যে যোগাযোগের জন্য আমরা যে বার্তাগুলি ব্যবহার করি তা হ্রাস করতে এবং আরো কিছু। আমরা এইসব ফিচারের উপরে কাজ করছি ফেডরেটেড লার্নিংকে উৎপাদন পরিবেশের জন্য প্রস্তুত করার জন্য এবং সেগুলো রিলিজ হওয়ার সাথে সাথে আমরা সেগুলো নিয়ে লিখবো।

আপনার এখন ফেডারেটেড লার্নিং নিজেই করতে সক্ষম হওয়া উচিত! আপনি যদি এটি উপভোগ করেন এবং গোপনীয়তা সংরক্ষণ, এআই এবং এআই সরবরাহ চেইনের (ডেটা) বিকেন্দ্রীভূত মালিকানার দিকে আন্দোলনে যোগ দিতে চান, আপনি নিম্নলিখিত উপায়ে এটি করতে পারেন!

### PySyft -কে গিটহাবে স্টার দিন

আমাদের কমিউনিটিকে সাহায্য করার সবচেয়ে সহজ পন্থা হলো রিপোজিটোরিতে স্টার দেয়া! এটি আমরা যে দারুন সরঞ্জাম তৈরি করছি সে ব্যাপারে সচেতনতা বৃদ্ধি করতে সাহায্য করে।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### গিটহাব থেকে আমাদের টিউটোরিয়ালগুলি বাছাই করুন!

ফেডারেটেড এবং প্রাইভেসি-প্রিজারভেভিং লার্নিংয়ের চেহারা কেমন হওয়া উচিত এবং আমরা এটির জন্য ইটগুলি কীভাবে তৈরি করছি সে সম্পর্কে আরও ভাল ধারণা পেতে আমরা সত্যিই দুর্দান্ত টিউটোরিয়াল তৈরি করেছি।

- [পাইসাইফ্ট টিউটোরিয়াল চেকআউট করুন](https://github.com/OpenMined/PySyft/tree/master/example/tutorials)


### আমাদের স্ল্যাকে(Slack) যোগ দিন!

সর্বশেষতম অগ্রগতিতে আপ টু ডেট রাখার সর্বোত্তম উপায় হলো আমাদের সম্প্রদায়ে যোগদান করা!

- [Join slack.openmined.org](http://slack.openmined.org)

### একটি কোড প্রকল্পে যোগদান করুন!

আমাদের কমিউনিটিতে অবদান রাখার সেরা উপায় হলো একজন কোড অবদানকারীতে পরিণত হওয়া। যেকোন সময় আপনি PySyft এর গিটহাব ইস্যুর পেজে যেতে পারেন এবং "Projects" দিয়ে বাছাই করবেন। এর মাধ্যমে আপনি যে সকল প্রজেক্টে যোগদান করতে পারবেন সেগুলোর উপরের দিকের টিকেটের ওভারভিউ পাবেন। আপনি যদি কোন প্রজেক্টে জয়েন করতে ইচ্ছুক না হোন, কিন্তু কিছুটা কোডিং করতে ইচ্ছুক সেক্ষেত্রে আপনি "one off" মিনি প্রজেক্টগুলো দেখতে পারেন গিটহাব ইস্যুতে "good first issue" চিহ্নিত ইস্যুগুলো।

- [good first issue](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করুন

আপনার যদি আমাদের কোডবেজে অবদান রাখারা সময় না হয়, কিন্তু এরপরও আমাদেরকে সমর্থন দিতে চান তাহলে আমাদের উন্মুক্ত সংগ্রহের সমর্থক হতে পারেন। সকল ধরনের দানের অর্থ আমাদের ওয়েব হোস্টিং এবং অন্যান্য কমিউনিটি কার্যক্রমে খরচ হয় যেমন - হ্যাকাথন, মিটাপ।

- [OpenMined's Open Collective Page](https://opencollective.com/openmined)